In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import sys

stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf8')
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde

def bbb(a,b):
    c = {}
    for i,j in eval(a).items():
        c[i.lower()] = j

    if b.lower() in c:
        if float(c[b.lower()])>0.1:
            return 1
        else:
            return 0
    else:
        return 0

train_data = pd.read_table('data/oppo_round1_train_20180929.txt',names=['prefix','query_prediction','title','tag','label'],header=None,encoding='utf-8').astype(str)
val_data = pd.read_table('data/oppo_round1_vali_20180929.txt',names=['prefix','query_prediction','title','tag','label'],header=None,encoding='utf-8').astype(str)
test_data = pd.read_table('data/oppo_round1_test_B_20181106.txt',names=['prefix','query_prediction','title','tag','label'],header=None,encoding='utf-8').astype(str)
train_data = pd.concat([train_data,pd.read_table('data26/oppo_round1_train_20180926.txt',names=['prefix','query_prediction','title','tag','label'],header=None,encoding='utf-8').astype(str)])
val_data = pd.concat([val_data,pd.read_table('data26/oppo_round1_vali_20180926.txt',names=['prefix','query_prediction','title','tag','label'],header=None,encoding='utf-8').astype(str)])
train_data = train_data[train_data['label']!='音乐']
train_data = pd.concat([train_data,val_data])
train_data['label'] = train_data['label'].apply(lambda x:int(x))
val_data['label'] = val_data['label'].apply(lambda x:int(x))
train_data['prefix_num'] = train_data['prefix'].apply(lambda x:len(x))
train_data['title_num'] = train_data['title'].apply(lambda x:len(x))
test_data['prefix_num'] = test_data['prefix'].apply(lambda x:len(x))
test_data['title_num'] = test_data['title'].apply(lambda x:len(x))

test_data['in_query_big'] = test_data.apply(lambda x:bbb(x['query_prediction'],x['title']),axis=1)


temp1 = train_data.groupby(['prefix','title','tag'],as_index=False)['label'].agg({'click1':'sum','count1':'count','ctr1':'mean'})
temp2 = train_data.groupby(['prefix','title'],as_index=False)['label'].agg({'click2':'sum','count2':'count','ctr2':'mean'})
temp3 = train_data.groupby(['prefix','tag'],as_index=False)['label'].agg({'click3':'sum','count3':'count','ctr3':'mean'})
temp4 = train_data.groupby(['title','tag'],as_index=False)['label'].agg({'click4':'sum','count4':'count','ctr4':'mean'})
temp5 = train_data.groupby('prefix',as_index=False)['label'].agg({'click5':'sum','count5':'count','ctr5':'mean'})
temp6 = train_data.groupby('title',as_index=False)['label'].agg({'click6':'sum','count6':'count','ctr6':'mean'})
temp7 = train_data.groupby('tag',as_index=False)['label'].agg({'click7':'sum','count7':'count','ctr7':'mean'})
temp8 = train_data.groupby(['prefix_num','title_num','tag'],as_index=False)['label'].agg({'click8':'sum','count8':'count','ctr8':'mean'})




a = pd.merge(test_data,temp1,on=['prefix','title','tag'],how='left')
b = pd.merge(test_data,temp2,on=['prefix','title'],how='left')
c = pd.merge(test_data,temp3,on=['prefix','tag'],how='left')
d = pd.merge(test_data,temp4,on=['title','tag'],how='left')
e = pd.merge(test_data,temp5,on='prefix',how='left')
f = pd.merge(test_data,temp6,on='title',how='left')
g = pd.merge(test_data,temp7,on='tag',how='left')
h = pd.merge(test_data,temp8,on=['prefix_num','title_num','tag'],how='left')


a['ctr2'] = b['ctr2']
a['ctr3'] = c['ctr3']
a['ctr4'] = d['ctr4']
a['ctr5'] = e['ctr5']
a['ctr6'] = f['ctr6']
a['ctr7'] = g['ctr7']
a['ctr8'] = h['ctr8']

a['count2'] = b['count2']
a['count3'] = c['count3']
a['count4'] = d['count4']
a['count5'] = e['count5']
a['count6'] = f['count6']
a['count7'] = g['count7']

a = a.fillna(0)
a['label2'] = 0

num1 = np.where((a['ctr1']>0.4),1,0)
num4 = np.where((a['ctr1']<=0.4)&(a['ctr1']>=0.2)&(a['ctr4']>0.5),1,0)
num7 = np.where((a['count1']==0)&(a['count4']==0)&(a['in_query_big']==0)&(a['ctr8']>0.5),1,0)
num8 = np.where((a['count1']==0)&(a['count4']==0)&(a['in_query_big']==1),1,0)
num9 = np.where((a['ctr4']>0.4)&(a['count1']==0),1,0)
num10 = np.where((a['count1']==0)&(a['ctr4']<=0.4)&(a['ctr4']>=0.2)&(a['in_query_big']==1),1,0)

numall2 = np.array(num1)+np.array(num4)+np.array(num8)+np.array(num7)+np.array(num9)+np.array(num10)


a['label3'] = numall2
print a.label3.mean()
# a['label3'].to_csv('sub/result.csv',index=False)

0.389964


In [5]:
temp2

,prefix,title,ctr2,count2,click2
0,#,#3,0.357143,14,5
1,#,#5,0.333333,15,5
2,#,#WYD,0.307692,13,4
3,%,%,0.312500,16,5
4,%,1%,0.545455,11,6
5,%,2%,1.000000,1,1
6,%,3%,0.200000,5,1
7,&,&gt;,0.250000,8,2
8,&,G&S,1.000000,1,1
9,&,H&U,1.000000,2,2


In [6]:
temp3

,prefix,tag,ctr3,count3,click3
0,#,百科,0.344828,29,10
1,#,音乐,0.307692,13,4
2,%,百科,0.393939,33,13
3,&,健康,0.250000,8,2
4,&,百科,0.666667,6,4
5,(女人,音乐,1.000000,1,1
6,*p++,知道,0.000000,1,0
7,-,健康,1.000000,1,1
8,.,健康,0.239130,92,22
9,.,百科,0.272085,283,77


In [10]:
train_data.loc[train_data.prefix == '#']

,prefix,query_prediction,title,tag,label,prefix_num,title_num
17464,#,"{""#*"": ""0.065"", ""#什么意思"": ""0.065"", ""#n/a是什么意思"":...",#WYD,音乐,0,1,4
85890,#,"{""#*"": ""0.065"", ""#什么意思"": ""0.065"", ""#n/a是什么意思"":...",#WYD,音乐,0,1,4
119412,#,"{""#*"": ""0.065"", ""#什么意思"": ""0.065"", ""#n/a是什么意思"":...",#3,百科,0,1,2
195122,#,"{""#*"": ""0.065"", ""#什么意思"": ""0.065"", ""#n/a是什么意思"":...",#WYD,音乐,0,1,4
258131,#,"{""#*"": ""0.065"", ""#什么意思"": ""0.065"", ""#n/a是什么意思"":...",#5,百科,0,1,2
260375,#,"{""#*"": ""0.065"", ""#什么意思"": ""0.065"", ""#n/a是什么意思"":...",#5,百科,0,1,2
270669,#,"{""#*"": ""0.065"", ""#什么意思"": ""0.065"", ""#n/a是什么意思"":...",#WYD,音乐,0,1,4
279169,#,"{""#*"": ""0.065"", ""#什么意思"": ""0.065"", ""#n/a是什么意思"":...",#5,百科,1,1,2
312525,#,"{""#*"": ""0.065"", ""#什么意思"": ""0.065"", ""#n/a是什么意思"":...",#5,百科,0,1,2
376013,#,"{""#*"": ""0.065"", ""#什么意思"": ""0.065"", ""#n/a是什么意思"":...",#5,百科,0,1,2


In [16]:
len(train_data.prefix.unique())

162753

In [17]:
len(train_data.query_prediction.unique())

158146

In [3]:
print 'a'

In [23]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import sys

stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf8')
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde

def bbb(a,b):
    c = {}
    for i,j in eval(a).items():
        c[i.lower()] = j

    if b.lower() in c:
        if float(c[b.lower()])>0.1:
            return 1
        else:
            return 0
    else:
        return 0

train_data = pd.read_table('data/oppo_round1_train_20180929.txt',names=['prefix','query_prediction','title','tag','label'],header=None,encoding='utf-8').astype(str)
val_data = pd.read_table('data/oppo_round1_vali_20180929.txt',names=['prefix','query_prediction','title','tag','label'],header=None,encoding='utf-8').astype(str)
test_data = pd.read_table('data/oppo_round1_test_B_20181106.txt',names=['prefix','query_prediction','title','tag','label'],header=None,encoding='utf-8').astype(str)
# train_data = pd.concat([train_data,pd.read_table('data26/oppo_round1_train_20180926.txt',names=['prefix','query_prediction','title','tag','label'],header=None,encoding='utf-8').astype(str)])
# val_data = pd.concat([val_data,pd.read_table('data26/oppo_round1_vali_20180926.txt',names=['prefix','query_prediction','title','tag','label'],header=None,encoding='utf-8').astype(str)])
train_data = train_data[train_data['label']!='音乐']
train_data = pd.concat([train_data,val_data])
train_data['label'] = train_data['label'].apply(lambda x:int(x))
val_data['label'] = val_data['label'].apply(lambda x:int(x))
train_data['prefix_num'] = train_data['prefix'].apply(lambda x:len(x))
train_data['title_num'] = train_data['title'].apply(lambda x:len(x))
test_data['prefix_num'] = test_data['prefix'].apply(lambda x:len(x))
test_data['title_num'] = test_data['title'].apply(lambda x:len(x))

test_data['in_query_big'] = test_data.apply(lambda x:bbb(x['query_prediction'],x['title']),axis=1)


temp1 = train_data.groupby(['prefix','title','tag'],as_index=False)['label'].agg({'click1':'sum','count1':'count','ctr1':'mean'})
temp2 = train_data.groupby(['prefix','title'],as_index=False)['label'].agg({'click2':'sum','count2':'count','ctr2':'mean'})
temp3 = train_data.groupby(['prefix','tag'],as_index=False)['label'].agg({'click3':'sum','count3':'count','ctr3':'mean'})
temp4 = train_data.groupby(['title','tag'],as_index=False)['label'].agg({'click4':'sum','count4':'count','ctr4':'mean'})
temp5 = train_data.groupby('prefix',as_index=False)['label'].agg({'click5':'sum','count5':'count','ctr5':'mean'})
temp6 = train_data.groupby('title',as_index=False)['label'].agg({'click6':'sum','count6':'count','ctr6':'mean'})
temp7 = train_data.groupby('tag',as_index=False)['label'].agg({'click7':'sum','count7':'count','ctr7':'mean'})
temp8 = train_data.groupby(['prefix_num','title_num','tag'],as_index=False)['label'].agg({'click8':'sum','count8':'count','ctr8':'mean'})

test_data = pd.merge(test_data,temp1,on=['prefix','title','tag'],how='left')
test_data = pd.merge(test_data,temp2,on=['prefix','title'],how='left')
test_data = pd.merge(test_data,temp3,on=['prefix','tag'],how='left')
test_data = pd.merge(test_data,temp4,on=['title','tag'],how='left')
test_data = pd.merge(test_data,temp5,on='prefix',how='left')
test_data = pd.merge(test_data,temp6,on='title',how='left')
test_data = pd.merge(test_data,temp7,on='tag',how='left')
test_data = pd.merge(test_data,temp8,on=['prefix_num','title_num','tag'],how='left')

test_data = test_data.fillna(0)
test_data['label2'] = 0


# a['label3'].to_csv('sub/result.csv',index=False)

In [72]:
num1 = np.where((test_data['ctr1']>0.4),1,0)
num4 = np.where((test_data['ctr1']<=0.4)&(test_data['ctr1']>=0.2)&(test_data['ctr4']>0.5),1,0)
num7 = np.where((test_data['count1']==0)&(test_data['count4']==0)&(test_data['in_query_big']==0)&(test_data['ctr8']>0.5),1,0)
num8 = np.where((test_data['count1']==0)&(test_data['count4']==0)&(test_data['in_query_big']==1),1,0)
num9 = np.where((test_data['ctr4']>0.4)&(test_data['count1']==0),1,0)
num10 = np.where((test_data['count1']==0)&(test_data['ctr4']<=0.4)&(test_data['ctr4']>=0.2)&(test_data['in_query_big']==1),1,0)

numall = np.array(num1)+np.array(num4)+np.array(num8)+np.array(num7)+np.array(num9)+np.array(num10)


test_data['label3'] = numall
test_data['label4'] = numall2
test_data['label5'] = test_data['label3']+test_data['label4']
print test_data.label5.value_counts()
print test_data.label3.mean()

0    148436
2     92144
1      9420
Name: label5, dtype: int64
0.3859


test_data['label3'].to_csv('sub/result.csv',index=False)

In [73]:
test_data['label3'].to_csv('sub/result.csv',index=False)